# Introduction

Data extracted from [Kaggle (The Movies Dataset)](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?select=ratings.csv).

In [14]:
# Load libraries
import pandas as pd
import re
from pyspark.sql.functions import regexp_extract, col, regexp_extract_all, lit, size
from helper_functions import *

# Initialize Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ETL").getOrCreate()
print(spark)

Files from the movies dataset:

In [2]:
import os
os.listdir("data")

['archive.zip',
 'cleaned',
 'credits.csv',
 'keywords.csv',
 'links.csv',
 'links_small.csv',
 'movies_metadata.csv',
 'movies_metadata_regex.csv',
 'ratings.csv',
 'ratings_small.csv']

## Ratings
This is the main dataset for the collaborative-filtering recommender system, as it contains opinions from the users about the movies.

In [3]:
df = load_and_show('data/ratings.csv', 
                   spark_session=spark, parquet=False)

Columns data types:


,Column Name,Data Type
0,userId,int
1,movieId,int
2,rating,double
3,timestamp,int


Number of partitions = 20
Number of entries/rows = 26024289

Data sample:


,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


Data description:


,summary,userId,movieId,rating,timestamp
0,count,26024289,26024289,26024289,26024289
1,mean,135037.090248114,15849.109677040553,3.5280903543608817,1.1712584326913226E9
2,stddev,78176.19722170144,31085.257531391675,1.0654427636662278,2.0528887028184557E8
3,min,1,1,0.5,789652004
4,max,270896,176275,5.0,1501829870


The column `timestamp` is not needed, because for a collaborative-filtering only the userId, itemId and rating are valuable. Let's write the full list of ratings without that column:

In [4]:
# Save cleaned ratings .csv file
df.drop('timestamp').\
    write.parquet('data/cleaned/ratings', mode='overwrite')

## Movies Metadata
This one contains lots of information about every movie in the dataset. However, when loading it for the first time, it was clear that there were reading problems. By checking the information in every column, it became apparent that extensive text entries caused them. The reasons are:

1. In some columns, like `overview`, there are commas, which are problematic when reading a file with comma-sepparated values. Nevertheless, when this happens the entry is contained between double quotation marks ("), so it is possible to avoid this wrong behaviour by specifying they indicate quotations.

2. These same texts include line breaks (\n) and, although it occures sparsely, it makes a single line be read as two or more. 

Let's start solving the second problem:

In [5]:
import re

# Open a new document to write the corrected lines
with open('data/movies_metadata_regex.csv', 'w+') as w:

    # Read the lines of movies_metadata.csv to locate and
    # correct the ones with line breaks
    with open('data/movies_metadata.csv', 'r') as r:
        # Skip first line of the iterable
        current_line = r.readline() # Header

        # Check every line
        for line in r:
            # Get first value of the line
            first_value = re.findall(r'^[^\,]*', line)[0]

            # Based on the first value, it is possible to know where there is a mistake or not
            # Correct values: True or False, in the 'Adult' column
            # Incorrect values: text from an overview or any other long text
            if first_value in ('False', 'True'):
                # Write resulting line:
                # + First one will be the header
                # + All \n are removed
                # + After that, include a \n at the end
                w.write(re.sub('\n', '', current_line) + '\n')

                # New line
                current_line = line
            else:
                # Append the line to the previous one
                current_line = current_line + line
        
        # Write last line
        w.write(re.sub('\n', '', current_line) + '\n')

Now, it is possible to read the .csv file by specifying the symbol " is the quotation character (already included in the helper function `load_and_show`):

In [6]:
df = load_and_show('data/movies_metadata_regex.csv',
                   spark_session=spark, parquet=False)

Columns data types:


,Column Name,Data Type
0,adult,boolean
1,belongs_to_collection,string
2,budget,int
3,genres,string
4,homepage,string
5,id,int
6,imdb_id,string
7,original_language,string
8,original_title,string
9,overview,string


Number of partitions = 9
Number of entries/rows = 45463

Data sample:


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,False,7.7,5415
1,False,None,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",None,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",None,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,None,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",None,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",None,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173


Data description:


,summary,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,production_companies,production_countries,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,count,4491,45463,45463,7779,45463,45446,45452,45463,44509,...,45463,45463,45463,45206,45463,45382,20413,45463,45463,45463
1,mean,None,4224578.813474693,None,None,108359.91881310077,None,None,Infinity,None,...,None,None,1.120860885984207E7,94.12757156129717,None,None,1942.0,Infinity,5.618217011635818,109.89085630072806
2,stddev,None,1.7424132634341218E7,None,None,112460.74927813222,None,None,NaN,None,...,None,None,6.433018853029212E7,38.406803036934555,None,None,None,NaN,1.9241741806737938,491.29481601544893
3,min,"{'id': 10, 'name': 'Star Wars Collection', 'po...",0,[],http://009.ph9.jp/,2,tt0000001,ab,!Women Art Revolution,,...,[],[],0,0.0,[],Canceled,,!Women Art Revolution,0.0,0
4,max,"{'id': 99727, 'name': 'Stuart Little Collectio...",380000000,"[{'id': 99, 'name': 'Documentary'}]",https://zeitgeistfilms.com/film/writerofo,469172,tt7158814,zu,ＳＭガールズ セイバーマリオネットＲ,"…ere erera baleibu izik subua aruaren…, a set ...",...,"[{'name': '프로덕션M', 'id': 19104}]","[{'iso_3166_1': 'ZA', 'name': 'South Africa'}]",2787965087,1256.0,"[{'iso_639_1': 'zu', 'name': 'isiZulu'}]",Rumored,李麗珍蜜桃成熟時,ファンタスティポ,10.0,14075


Not only did it get the right structure, but it also infered the correct schema, which indicates the correction was successful.

Another issue with this data is the presence of JSON files in some of the columns (specifically, lists of JSON files). This is the case for:
+ `belongs to collection`.
+ `genres`.
+ `production_companies`.
+ `production_countries`.
+ `spoken_languages`.

Instead of that, it can be transformed into a list of items (genres, languages, etc.).

On the other hand, there are several columns that don't have much of an impact on the analysis:
+ `homepage`.
+ `imdb_id`.
+ `poster_path`.
+ `original_title`.

This can easily be solved with the next code:

In [7]:
# Drop useless columns
df = df.drop('homepage', 'imdb_id', 'poster_path', 'original_title')
df.show(5)

+-----+---------------------+--------+--------------------+-----+-----------------+--------------------+----------+--------------------+--------------------+------------+---------+-------+--------------------+--------+--------------------+--------------------+-----+------------+----------+
|adult|belongs_to_collection|  budget|              genres|   id|original_language|            overview|popularity|production_companies|production_countries|release_date|  revenue|runtime|    spoken_languages|  status|             tagline|               title|video|vote_average|vote_count|
+-----+---------------------+--------+--------------------+-----+-----------------+--------------------+----------+--------------------+--------------------+------------+---------+-------+--------------------+--------+--------------------+--------------------+-----+------------+----------+
|false| {'id': 10194, 'na...|30000000|[{'id': 16, 'name...|  862|               en|Led by Woody, And...| 21.946943|[{'name': 'P

Now, let's focus on the challenging part, finding a way to transform a string that represents a list of JSON files into a list of items. One way to do this is with regular expressions, which can be applied row wise with PySpark.

Let's take a sample to check if this approach is valid:

In [8]:
sample = df.limit(3).\
    select('belongs_to_collection', 'genres', 
           'production_companies', 'production_countries',
           'spoken_languages').\
    toPandas()
    
display(sample)

,belongs_to_collection,genres,production_companies,production_countries,spoken_languages
0,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]"
1,None,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso..."
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]"


+ `belongs_to_collection`: the only important piece of information is the name of the collection (stored under 'name' field).

+ `genres`: it is a list of JSON files. Each of them contains the field 'name', which indicates one of the genres associated with the movie.

+ `production_companies`: it is a list of JSON files. The names of the companies are stored under the field 'name', similar to `genres`.

+ `production_countries`: it is a list of JSON files. Again, the country name is stored under the field 'name', but the abbreviation for it is also accesible (in this case, under the field 'iso_3166_1').

+ `spoken_languages`: it is a list of JSON files. It is the same as `production_countries`, except the abbreviation is stored under 'iso_639_1'.

In [9]:
# Example: extract information under the field 'name'
l_json = sample.loc[0,'spoken_languages']
re.findall(
    pattern=r'(?<=\'name\': \')([^\']*)(?=\')',
    string=l_json
)

['English']

In [10]:
# Example: extract information under the field 'iso_3166_1'
l_json = sample.loc[0,'production_countries']
re.findall(
    pattern=r'(?<=\'iso_3166_1\': \')([^\']*)(?=\')',
    string=l_json
)

['US']

In [11]:
# Example: extract information under the field 'iso_639_1'
l_json = sample.loc[0,'spoken_languages']
re.findall(
    pattern=r'(?<=\'iso_639_1\': \')([^\']*)(?=\')',
    string=l_json
)

['en']

All that is left to do is extract this information for the full column using PySpark:

In [12]:
# Collection: a single collection for each movie (not a list of JSONs)
df = df.withColumn(
    'collection', 
    regexp_extract(col('belongs_to_collection'), 
        r'(?<=\'name\': \')([^\']*)(?=\')', 
        1)
    ).\
    drop('belongs_to_collection')
df.show(5)

+-----+--------+--------------------+-----+-----------------+--------------------+----------+--------------------+--------------------+------------+---------+-------+--------------------+--------+--------------------+--------------------+-----+------------+----------+--------------------+
|adult|  budget|              genres|   id|original_language|            overview|popularity|production_companies|production_countries|release_date|  revenue|runtime|    spoken_languages|  status|             tagline|               title|video|vote_average|vote_count|          collection|
+-----+--------+--------------------+-----+-----------------+--------------------+----------+--------------------+--------------------+------------+---------+-------+--------------------+--------+--------------------+--------------------+-----+------------+----------+--------------------+
|false|30000000|[{'id': 16, 'name...|  862|               en|Led by Woody, And...| 21.946943|[{'name': 'Pixar ...|[{'iso_3166_1': 

In [13]:
# Columns with lists of JSONs (list of items)
df = df.\
    withColumn(
        'genres', 
        regexp_extract_all('genres', 
            lit(r'(?<=\'name\': \')([^\']*)(?=\')').alias('d'))   
    ).\
    withColumn(
        'production_companies', 
        regexp_extract_all('production_companies', 
            lit(r'(?<=\'name\': \')([^\']*)(?=\')').alias('d'))   
    ).\
    withColumn(
        'production_countries', 
        regexp_extract_all('production_countries', 
            lit(r'(?<=\'name\': \')([^\']*)(?=\')').alias('d'))   
    ).\
    withColumn(
        'spoken_languages', 
        regexp_extract_all('spoken_languages', 
            lit(r'(?<=\'name\': \')([^\']*)(?=\')').alias('d'))   
    )

display(df.limit(5).toPandas())

,adult,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection
0,False,30000000,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033,81.0,[English],Released,None,Toy Story,False,7.7,5415,Toy Story Collection
1,False,65000000,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413,None
2,False,0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92,Grumpy Old Men Collection
3,False,16000000,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,81452156,127.0,[English],Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34,None
4,False,0,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,76578911,106.0,[English],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173,Father of the Bride Collection


In [17]:
for column in ['genres', 'production_companies', 'production_countries', 'spoken_languages']:
    print('n_' + column)
    df = df.withColumn('n_' + column, size(df[column]))

df.show()

n_genres
n_production_companies
n_production_countries
n_spoken_languages
+-----+--------+--------------------+-----+-----------------+--------------------+----------+--------------------+--------------------+------------+---------+-------+--------------------+--------+--------------------+--------------------+-----+------------+----------+--------------------+--------+----------------------+----------------------+------------------+
|adult|  budget|              genres|   id|original_language|            overview|popularity|production_companies|production_countries|release_date|  revenue|runtime|    spoken_languages|  status|             tagline|               title|video|vote_average|vote_count|          collection|n_genres|n_production_companies|n_production_countries|n_spoken_languages|
+-----+--------+--------------------+-----+-----------------+--------------------+----------+--------------------+--------------------+------------+---------+-------+--------------------+--------+--

Additionally, `status` and `original_language` may be treated as bool, depending on their uniques values:

In [18]:
df.\
    select(['status']).distinct().\
    show()

df.\
    select(['original_language']).distinct().\
    show()

+---------------+
|         status|
+---------------+
|        Rumored|
|       Released|
|Post Production|
|  In Production|
|        Planned|
|       Canceled|
|           NULL|
+---------------+

+-----------------+
|original_language|
+-----------------+
|               en|
|               vi|
|               nb|
|               ro|
|               pl|
|               cn|
|               sk|
|               pt|
|               sh|
|               ko|
|               cs|
|               sr|
|               de|
|               es|
|               el|
|               it|
|               ar|
|               sv|
|               nl|
|               bo|
+-----------------+
only showing top 20 rows



It is not the case. Finally, we save the cleaned data in parquet format:

In [19]:
df.write.parquet('data/cleaned/movies_metadata', mode='overwrite')

## Links

In [16]:
df = load_and_show('data/links.csv')

root
 |-- movieId: integer (nullable = true)
 |-- imdbId: integer (nullable = true)
 |-- tmdbId: integer (nullable = true)

Number of partitions = 1
Number of entries/rows = 458430

+-------+------+------+
|movieId|imdbId|tmdbId|
+-------+------+------+
|      1|114709|   862|
|      2|113497|  8844|
|      3|113228| 15602|
|      4|114885| 31357|
|      5|113041| 11862|
+-------+------+------+
only showing top 5 rows



For now, this doesn't seem to be useful. Let's leave it be.

## Keywords

In [44]:
df = load_and_show('data/keywords.csv')

root
 |-- id: integer (nullable = true)
 |-- keywords: string (nullable = true)

Number of partitions = 2
Number of entries/rows = 464190

+-----+--------------------+
|   id|            keywords|
+-----+--------------------+
|  862|[{'id': 931, 'nam...|
| 8844|[{'id': 10090, 'n...|
|15602|[{'id': 1495, 'na...|
|31357|[{'id': 818, 'nam...|
|11862|[{'id': 1009, 'na...|
+-----+--------------------+
only showing top 5 rows



It is being read correctly and the keywords have the potential to be used as vocabulary for NLP techniques. However, if the keywords' ids are to be trusted, it would require a number of words that far exceeds what my resources can manage (at least 30000 words! That would translate into a number_movies $\times$ 30000 matrix for methods like TF-IDF).

Let's take a sample to look for a way to extract the keywords:

In [45]:
import pandas as pd
pd_df = df.limit(5).toPandas()
pd_df.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [46]:
import re
l_json = pd_df.iloc[0,1]
re.findall(
    pattern=r'(?<=\'name\': \')([^\']*)(?=\')',
    string=l_json
)

['jealousy',
 'toy',
 'boy',
 'friendship',
 'friends',
 'rivalry',
 'boy next door',
 'new toy',
 'toy comes to life']

Now, an option is to save this list of keywords in the dataframe (like the original list of json's), which should be more accesible:

In [53]:
from pyspark.sql.functions import regexp_extract_all, lit

df = df.withColumn(
    'keywords_list', 
    regexp_extract_all('keywords', 
        lit(r'(?<=\'name\': \')([^\']*)(?=\')').alias('d'))
    ).drop('keywords')
df.show(5)

+-----+--------------------+
|   id|       keywords_list|
+-----+--------------------+
|  862|[jealousy, toy, b...|
| 8844|[board game, disa...|
|15602|[fishing, best fr...|
|31357|[based on novel, ...|
|11862|[baby, midlife cr...|
+-----+--------------------+
only showing top 5 rows



Write as parquet:

In [54]:
df.write.parquet('data/cleaned/keywords', mode='overwrite')

## Credits


In [36]:
df = load_and_show('data/credits.csv')

root
 |-- cast: string (nullable = true)
 |-- crew: string (nullable = true)
 |-- id: integer (nullable = true)

Number of partitions = 20
Number of entries/rows = 454760

+--------------------+--------------------+-----+
|                cast|                crew|   id|
+--------------------+--------------------+-----+
|[{'cast_id': 14, ...|[{'credit_id': '5...|  862|
|[{'cast_id': 1, '...|[{'credit_id': '5...| 8844|
|[{'cast_id': 2, '...|[{'credit_id': '5...|15602|
|[{'cast_id': 1, '...|[{'credit_id': '5...|31357|
|[{'cast_id': 1, '...|[{'credit_id': '5...|11862|
+--------------------+--------------------+-----+
only showing top 5 rows



The 'crew' column is specially interesting, as the actors and the directors are mentioned. They may be used in a content-based recommender system, specially the director.

Let's analyse a subset to find a way to extract the director of each movie:

In [37]:
import pandas as pd
pd_df = df.limit(5).toPandas()
pd_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [38]:
import re
l_json = pd_df.iloc[0,1]
re.findall(
    pattern=r'(?<=Director\', \'name\': \')([^\']*)(?=\')',
    string=l_json
)

['John Lasseter', 'Ash Brannon', 'William Reeves', 'Rich Quade']

Now, let's use PySpark to parallelize this operation when applying it to the `crew` column:

In [39]:
from pyspark.sql.functions import regexp_extract, col

df = df.withColumn(
    'director', 
    regexp_extract(col('crew'), 
        r'(?<=Director\', \'name\': \')([^\']*)(?=\')', 
        1)
    ).\
    drop('cast', 'crew')
df.show(5)

+-----+---------------+
|   id|       director|
+-----+---------------+
|  862|  John Lasseter|
| 8844|   Joe Johnston|
|15602|  Howard Deutch|
|31357|Forest Whitaker|
|11862|  Charles Shyer|
+-----+---------------+
only showing top 5 rows



Let's save the director for each movie:

In [21]:
df.write.parquet('data/cleaned/directors', mode='overwrite')

Close Spark instance:

In [20]:
spark.stop()

## References

+ [The Movies DataSet](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?select=ratings.csv)

+ [Advances PySpark for EDA](https://www.kaggle.com/code/tientd95/advanced-pyspark-for-exploratory-data-analysis#1.-Initialize-pyspark-framework-and-load-data-into-pyspark's-dataframe-)

+ [Spark Sessions](https://sparkbyexamples.com/spark/sparksession-explained-with-examples/)

+ [List from Regular Expression](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.regexp_extract_all.html)
